In [3]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.8 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [4]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-01 06:50:04--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-03-01 06:50:05 (10.6 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [7]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

auto_df = spark.read.option("header", "true").option("sep", "\t").csv("file://" + SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"))
auto_df.show(10)



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [18]:
# Count the number of records (rows) in the dataset.
auto_df.count()

3514942

In [42]:
auto_df.dropna()

DataFrame[marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string, customer_count: int]

In [43]:
# Find the total amount of customers who reviewed.

cust_groupby = auto_df.groupBy("customer_id").count()
cust_groupby.orderBy("customer_id").select("customer_id", "count").show()



+-----------+-----+
|customer_id|count|
+-----------+-----+
|   10000029|    1|
|    1000004|    2|
|   10000051|    1|
|   10000062|    1|
|   10000063|    1|
|   10000071|    2|
|   10000088|    1|
|   10000094|    1|
|   10000153|    2|
|   10000200|    1|
|   10000206|    2|
|   10000226|    1|
|   10000251|    1|
|   10000264|    1|
|   10000277|    1|
|   10000284|    1|
|    1000029|    3|
|   10000294|    9|
|   10000297|    1|
|   10000313|    1|
+-----------+-----+
only showing top 20 rows



In [44]:
cust_groupby.select(["customer_id"]).describe().show()

+-------+--------------------+
|summary|         customer_id|
+-------+--------------------+
|  count|             1907652|
|   mean|2.5359115781779382E7|
| stddev|  1.57403020913828E7|
|    min|            10000029|
|    max|             9999955|
+-------+--------------------+



In [45]:
from pyspark.sql.functions import lit



auto_df = auto_df.withColumn('customer_count', lit(1907652))


# customers = auto_df.select(col("customer_id"),lit("1907652").as("customer_count")
# customers.show()

# from pyspark.sql.functions import lit
 
# df.withColumn('new_column', lit(10))


In [46]:
# Create user dataframe to match active_user table
review_id_table = auto_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RAB23OVFNCXZQ|   36075342|B00LPRXQ4Y|     339193102| 2015-08-31|
|R3NORADVJO6IE6|   42462164|B000C7S0TO|     907684644| 2015-08-31|
|R299F4SO98S5OO|   21241933|B000CO9WE4|     752246352| 2015-08-31|
|R2DA9DOT03UW6I|   52570308|B000GKD5NI|     105401756| 2015-08-31|
|R2OGCH681EQHU6|   38200102|B009SDA7TE|     728471129| 2015-08-31|
| R2JMKIC16MHD7|   34866169|B00KV15KRG|     962286893| 2015-08-31|
|R1DB5DA7CWWTI8|     184627|B0002JMAKW|     267002949| 2015-08-31|
|R1N8XWFDK4QACP|    2975964|B00XJKMM6S|     570789093| 2015-08-31|
|R19VFLX6MRU0CX|   40946484|B000C5CEKC|     389524802| 2015-08-31|
| RLTD8YDGTP9PT|   12554469|B00GD9R2A8|      21425394| 2015-08-31|
| R78B5LCFJ52SR|   35335277|B00RM327NG|     816815445| 2015-08-31|
|R33SKWL0HEQIQ9|   44957003|B0071NL7UQ|     192225755| 2015-08

In [63]:
# Create user dataframe to match products table

# products = auto_df.dropDuplicates((["product_id", "product_title"])).select("product_id", "product_title")
# products.show()

products = auto_df.select(["product_id", "product_title"]).distinct()
products

DataFrame[product_id: string, product_title: string]

In [48]:
# Create user dataframe to match customers table
customers = auto_df.select(["customer_id", "customer_count"])
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   36075342|       1907652|
|   42462164|       1907652|
|   21241933|       1907652|
|   52570308|       1907652|
|   38200102|       1907652|
|   34866169|       1907652|
|     184627|       1907652|
|    2975964|       1907652|
|   40946484|       1907652|
|   12554469|       1907652|
|   35335277|       1907652|
|   44957003|       1907652|
|     444125|       1907652|
|   35829199|       1907652|
|   16825098|       1907652|
|    2583392|       1907652|
|   10127903|       1907652|
|   17937309|       1907652|
|   17145638|       1907652|
|   32813839|       1907652|
+-----------+--------------+
only showing top 20 rows



In [25]:
# Create user dataframe to match vine table
vine = auto_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RAB23OVFNCXZQ|          1|            0|          0|   N|
|R3NORADVJO6IE6|          5|            0|          0|   N|
|R299F4SO98S5OO|          5|            1|          1|   N|
|R2DA9DOT03UW6I|          5|            2|          3|   N|
|R2OGCH681EQHU6|          5|            0|          0|   N|
| R2JMKIC16MHD7|          5|            2|          2|   N|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|
|R1N8XWFDK4QACP|          5|            0|          0|   N|
|R19VFLX6MRU0CX|          5|            0|          0|   N|
| RLTD8YDGTP9PT|          1|            0|          0|   N|
| R78B5LCFJ52SR|          4|            0|          0|   N|
|R33SKWL0HEQIQ9|          5|            0|          0|   N|
|R1HIOV0UX7P13D|          5|            0|          0|   N|
| R4WS0E0MSP9DH|          5|            

In [61]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://XXXXXXXXXXX:5432/big_data_challenge_RDS_1"
config = {"user":"XXXXXXXXX",
          "password": "XXXXXXXXXXX",
          "driver":"org.postgresql.Driver"}

In [50]:
# Write review_id_table DataFrame to table

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [64]:
# Write products DataFrame to table

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [52]:
# Write products DataFrame to table

customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [35]:
# Write vine DataFrame to table

vine.write.jdbc(url=jdbc_url, table='vine', mode=mode, properties=config)